<h1><b>Intro to Cluster Computing</b></h1><br/>
   - What and why
<br/><br/>
<h1><b>Hadoop</b></h1><br/>
   - What and why, as well as limitations
<br/><br/>
<h1><b>Spark</b></h1><br/>
   - How it overcomes Hadoop's limitations

<h1><b>Introduction to RDD's.</b></h1><br/>
An RDD is short for Resilient Distributed Dataset. We can create an RDD with Spark, and it will be broken up into chunks and stored on multiple devices. We can then perform parallelized operations on RDD's almost as if it were a single, local, file on one device, without having to worry about the details of multi-processing. This is the power of Spark. This results in very elegant, intuitive source code.
<br/><br/>
For example, we can create a simple RDD with the following code:


In [ ]:
from pyspark import SparkContext
sc = sc.SparkContext("Local", "My App")
data = sc.textFile("example.txt")
print data.collect() #Show that data


Problem 1: Create an RDD from the textfile nums.txt, and print it, following the example code above. Save your code in a file nums.py, as you will be performing further operations on the dataset nums.txt.

Solution:

In [1]:
from pyspark import SparkContext
sc = SparkContext("local", "My App")
data = sc.textFile("nums.txt")


SyntaxError: invalid syntax (<ipython-input-1-8e4d9588c430>, line 2)

Problem 2: RDD Actions.
Modify the nums.py file you created in Problem 1 to print the following statistics about the dataset.<br/>
(i) Number of lines in the RDD. <br/>
(ii) The first 5 elements of the dataset.<br/>
(iii) The lexicographically first string in the dataset. (HINT: use reduce with max)<br/>
(iv) The entire dataset, as an array.<br/>
<br/>
Solution:

In [ ]:
from pyspark import SparkContext
sc = SparkContext("local", "My App")
data = sc.textFile("nums.txt")
print "Num Lines: %d" % data.count()
print data.take(5)
print data.reduce(max)
print data.collect()

Problem 3: RDD transformations<br/>
Further extend your file, nums.py to answer the following questions about your dataset, using RDD transformations (filter, reduce, map).<br/>
(i) The number of numbers less than 50.<br/>
(ii) The sum of the numbers in the dataset.<br/>
(iii) The sum of the squares of the dataset.<br/><br/>
Solutions:

In [ ]:
from pyspark import SparkContext
from math import log

sc = SparkContext("local", "My App")
data = sc.textFile("nums.txt")
print "Num Lines: %d" % data.count()
print "First five elements.", data.take(5)
print "Maximum Element " + data.reduce(max)
print "Total dataset ", data.collect()

intData = data.map(int)
print "Numbers less than 50: %d" % intData.filter(lambda a: a < 50).count()
print "Sum %d " % intData.reduce(lambda a,b : a+b)
print "Sum Of Squares %d" % intData.map(lambda x: x*x).reduce(lambda a,b: a+b)

Problem 4: Basic Map-Reduce<br/>
Create a new file, words.py. Create an RDD from words.txt. By following the steps below, use MapReduce to compute the number of words start with each letter.<br/>
(i) Create a mapper function map that maps a word to a key-value pair where the key is the first letter of the word, and the value is simply 1, as in the preceding example. (TODO write an example)<br/>
(ii) Create a reducer function that simply sums two values.<br/>
(iii) Using the operations map, reduceByKey, and collect, determine, for each letter, the number of words that start with that letter.<br/><br/>
Solutions

In [ ]:
from pyspark import SparkContext

corpus = "words.txt" 
sc = SparkContext("local", "Simple App")
#Create the RDD
data = sc.textFile(corpus).cache()


def mapper(w):
  return (w[0], 1)

def reducer(a,b):
 return a+b

print data.map(mapper).reduceByKey(reducer).collect()

Problem 5: Advanced Map-Reduce with bigrams.<br/> Extend the words.py file you made to count the number of bigrams in 
(i)<br/>
(ii)<br/>

The following program reads in a file of words (words.txt